In [53]:
import numpy as np
import pandas as pd

from empiricaldist import Pmf

import src.helpers as hlp
from src.helpers import prob, conditional

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Probability

## Problem 1

In [84]:
# Download and read data
hlp.download('https://github.com/AllenDowney/ThinkBayes2/raw/master/data/gss_bayes.csv')
gss = pd.read_csv('data/gss_bayes.csv', index_col=0)
gss.head(3)

,year,age,sex,polviews,partyid,indus10
caseid,,,,,,
1,1974,21.0,1,4.0,2.0,4970.0
2,1974,41.0,1,5.0,0.0,9160.0
5,1974,58.0,2,6.0,1.0,2670.0


In [85]:
# Create required variables
female = gss['sex'] == 2
banker = gss['indus10'] == 6870
liberal = gss['polviews'] <= 3
democrat = gss['partyid'] <= 1
is31 = gss['age'] == 31

In [86]:
p = prob(is31 & female & banker)
print(f"P(31 and female and banker) = {p:.5f}")

P(31 and female and banker) = 0.00026


In [77]:
p = prob(is31 & female & banker & democrat & liberal)
print(f"P(31 and female and banker and liberal and democrat) = {p:.4f}")

P(31 and female and banker and liberal and democrat) = 0.0001


## Problem 2

In [79]:
p = liberal[democrat].mean()
assert p == conditional(liberal, democrat)
print(f"P(liberal|democrat) = {p:.3f}")

P(liberal|democrat) = 0.389


In [80]:
p = democrat[liberal].mean()
print(f"P(demo|liberal) = {p:.3f}")

P(demo|liberal) = 0.521


## Problem 3

In [88]:
# Create required variables
young = gss.age < 30
old = gss.age >= 65
conservative = gss.polviews >= 5


In [97]:
p = prob(young & liberal)
print('P(young & liberal) =', p)

P(young & liberal) = 0.06579427875836884


In [93]:
p = prob(liberal[young])
print('P(liberal|young) =', p)

P(liberal|young) = 0.338517745302714


In [96]:
p = prob(conservative & old)
print('p(conservative & old) =', p)

p(conservative & old) = 0.06701156421180766


In [99]:
p = prob(old[conservative])
print('P(old|conservative) =', p)

P(old|conservative) = 0.19597721609113564
